**Weight Optimization Using Pso**

In [ ]:
import pandas as pd
from pyswarm import pso  # Install this package: pip install pyswarm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm  # Import tqdm for progress bar
from sklearn.impute import SimpleImputer  # Import imputer to handle missing values

# Step 1: Load and Standardize Column Names
file_paths = [
    r"new dataset\DWT\DWT_wavelet_features.csv",
    r"new dataset\new dataset\GLCM\glcm_features.csv",
    r"new dataset\new dataset\swt\sWT_features_with_labels.csv",
    r"new dataset\new dataset\swt+glcm\swt_glcm_features.csv",
    r"new dataset\new dataset\dwt_glcm_features.csv",
    r"new dataset\new dataset\SWT+DWT+GLCMcombined_features.csv",
    r"new dataset\new dataset\cnn\CNN_features.csv" 
]

feature_sets = []
meta_data = None  # To store 'filename' and 'category' once, assuming they are consistent


# Load the datasets and preprocess
for path in tqdm(file_paths, desc="Loading Datasets", unit="file"):
    df = pd.read_csv(path)
    
    # Rename columns for consistency
    df.rename(columns={'Filename': 'filename', 'Labels': 'category'}, inplace=True)
    
    # Separate metadata (filename and category) from features
    if meta_data is None:
        meta_data = df[['filename', 'category']]
    else:
        df = df.drop(columns=['filename', 'category'], errors='ignore')
    
    # Add to feature sets list
    feature_sets.append(df)

# Step 2: Merge Feature Sets
merged_features = meta_data.copy()

for i, features in tqdm(enumerate(feature_sets), desc="Merging Feature Sets", total=len(feature_sets)):
    features = features.add_prefix(f"set{i}_")
    merged_features = pd.concat([merged_features, features], axis=1)

# Step 3: Prepare Data for Optimization
labels = merged_features['category']
features = merged_features.drop(columns=['filename', 'category'], errors='ignore')

# Encode labels for classification
labels = pd.factorize(labels)[0]

# Split into training and testing data
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

def objective_function(weights):
    weighted_features_train = 0
    weighted_features_test = 0

    # Apply weights to feature sets
    for i in range(len(feature_sets)):
        train_set = X_train.filter(like=f'set{i}_')
        test_set = X_test.filter(like=f'set{i}_')

        # Convert to numeric if necessary
        train_set = train_set.apply(pd.to_numeric, errors='coerce')
        test_set = test_set.apply(pd.to_numeric, errors='coerce')

        # Apply weights
        weighted_features_train += train_set * weights[i]
        weighted_features_test += test_set * weights[i]

    # Handle missing values by filling NaNs with the column mean
    imputer = SimpleImputer(strategy='mean')
    weighted_features_train = imputer.fit_transform(weighted_features_train)
    weighted_features_test = imputer.transform(weighted_features_test)

    # Ensure no NaN values exist in the training and testing datasets after imputation
    if np.isnan(weighted_features_train).any() or np.isnan(weighted_features_test).any():
        print("Warning: Missing values detected after applying weights.")

    # Check if weighted features are empty
    if weighted_features_train.shape[1] == 0:
        print("Warning: Weighted feature set is empty for training.")
        return 1  # Return a high value to avoid optimization with empty features

    # Train logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(weighted_features_train, y_train)
    
    # Predict on test set
    predictions = model.predict(weighted_features_test)
    
    # Calculate accuracy (maximize this, so return -accuracy)
    accuracy = accuracy_score(y_test, predictions)
    return -accuracy  # Minimize negative accuracy for optimization

# Step 4: Optimize Weights for Feature Sets
num_feature_sets = len(feature_sets)
lb = [0.1] * num_feature_sets  # Lower bounds for weights
ub = [1.0] * num_feature_sets  # Upper bounds for weights

print("Starting PSO optimization...")

# Perform PSO optimization
best_weights, best_accuracy = pso(objective_function, lb, ub, swarmsize=30, maxiter=100)

# Step 5: Save and Print Optimized Weights
weights_output_path = r"new dataset\optimized\optimized_feature_set_weights.csv"
weights_df = pd.DataFrame({
    'Feature_Set': [f"set{i}" for i in range(num_feature_sets)],
    'Weight': best_weights
})
weights_df.to_csv(weights_output_path, index=False)

print(f"Optimized weights saved to: {weights_output_path}")
print(f"Best Accuracy Achieved: {-best_accuracy}")

# Display Final Optimized Weights
print("Final Optimized Weights for Each Feature Set:")
for i, weight in enumerate(best_weights):
    print(f"set{i}: {weight:.4f}")


**Classifiers training on obtained weights by PSO**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC

# Load weights
weights_path = r"new dataset\new dataset\optimized\optimized_feature_set_weights.csv"  # Update with your weights file path
weights = pd.read_csv(weights_path)["Weight"].values

# Load and combine feature sets
feature_paths = [
    r"new dataset\new dataset\DWT\DWT_wavelet_features.csv",
    r"new dataset\new dataset\GLCM\glcm_features.csv",
    r"new dataset\new dataset\swt\sWT_features_with_labels.csv",
    r"new dataset\new dataset\swt+glcm\swt_glcm_features.csv",
    r"new dataset\new dataset\dwt_glcm_features.csv",
    r"new dataset\new dataset\SWT+DWT+GLCMcombined_features.csv",
    r"new dataset\new dataset\cnn\CNN_features.csv"
]

feature_sets = [pd.read_csv(path) for path in feature_paths]

# Extract labels and preprocess features
labels = feature_sets[0]["category"]  # Assuming all files share the same labels
for i in range(len(feature_sets)):
    feature_sets[i] = feature_sets[i].drop(columns=["filename", "category"], errors="ignore")
    feature_sets[i] = feature_sets[i].apply(pd.to_numeric, errors='coerce')  # Convert to numeric
    feature_sets[i] = feature_sets[i].fillna(0)  # Fill NaN with 0

# Apply weights to features
weighted_features = None
for i, (features, weight) in enumerate(zip(feature_sets, weights)):
    weighted_set = features.add_prefix(f"set{i}_") * weight  # Apply weight
    weighted_features = pd.concat([weighted_features, weighted_set], axis=1) if weighted_features is not None else weighted_set

# Normalize features
scaler = RobustScaler()
weighted_features = scaler.fit_transform(weighted_features)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(weighted_features, encoded_labels, test_size=0.2, random_state=42, stratify=encoded_labels)

# Train the SVM model
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X_train, y_train)

# Calculate training accuracy
y_train_pred = svm_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Calculate testing accuracy
y_test_pred = svm_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Testing Accuracy: {test_accuracy:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred, target_names=label_encoder.classes_))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

# Load weights
weights_path = r"new dataset\new dataset\optimized\optimized_feature_set_weights.csv"  # Update with your weights file path
weights = pd.read_csv(weights_path)["Weight"].values

# Load and combine feature sets
feature_paths = [
    r"new dataset\new dataset\DWT\DWT_wavelet_features.csv",
    r"new dataset\new dataset\GLCM\glcm_features.csv",
    r"new dataset\new dataset\swt\sWT_features_with_labels.csv",
    r"new dataset\new dataset\swt+glcm\swt_glcm_features.csv",
    r"new dataset\new dataset\dwt_glcm_features.csv",
    r"new dataset\new dataset\SWT+DWT+GLCMcombined_features.csv",
    r"new dataset\new dataset\cnn\CNN_features.csv"
]

feature_sets = [pd.read_csv(path) for path in feature_paths]

# Extract labels and remove unnecessary columns
labels = feature_sets[0]["category"]  # Assuming all files share the same labels
for i in range(len(feature_sets)):
    feature_sets[i] = feature_sets[i].drop(columns=["filename", "category"], errors="ignore")
    feature_sets[i] = feature_sets[i].apply(pd.to_numeric, errors='coerce')  # Convert to numeric
    feature_sets[i] = feature_sets[i].fillna(0)  # Fill NaN with 0

# Apply weights to features
weighted_features = None
for i, (features, weight) in tqdm(enumerate(zip(feature_sets, weights)), desc="Applying Weights", total=len(feature_sets)):
    weighted_set = features.add_prefix(f"set{i}_") * weight  # Apply weight
    weighted_features = pd.concat([weighted_features, weighted_set], axis=1) if weighted_features is not None else weighted_set

# Ensure data alignment
assert len(weighted_features) == len(labels), "Features and labels must have the same number of samples."

# Normalize features
scaler = RobustScaler()
weighted_features = scaler.fit_transform(weighted_features)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(np.unique(encoded_labels))
y_categorical = to_categorical(encoded_labels, num_classes=num_classes)

# Reshape features for CNN input (assuming 1D convolution)
X_train, X_test, y_train, y_test = train_test_split(weighted_features, y_categorical, test_size=0.2, random_state=42, stratify=encoded_labels)
X_train = X_train[..., np.newaxis]  # Add channel dimension for CNN
X_test = X_test[..., np.newaxis]

# Simplified CNN model to reduce memory consumption
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train with smaller batch size
history = model.fit(X_train, y_train, epochs=150, batch_size=16, validation_split=0.2, verbose=2)

# Evaluate the model
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")


# Predictions and classification report
y_test_pred = model.predict(X_test)
y_test_pred_classes = np.argmax(y_test_pred, axis=1)
y_test_true_classes = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(y_test_true_classes, y_test_pred_classes, target_names=label_encoder.classes_))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from tqdm import tqdm

# File paths for feature vectors and weights
file_paths = [
    r"new dataset\new dataset\DWT\DWT_wavelet_features.csv",
    r"new dataset\new dataset\GLCM\glcm_features.csv",
    r"new dataset\new dataset\swt\sWT_features_with_labels.csv",
    r"new dataset\new dataset\swt+glcm\swt_glcm_features.csv",
    r"new dataset\new dataset\dwt_glcm_features.csv",
    r"new dataset\new dataset\SWT+DWT+GLCMcombined_features.csv",
    r"new dataset\new dataset\cnn\CNN_features.csv"
]

weights_csv_path = r"new dataset\new dataset\optimized\optimized_feature_set_weights.csv"

# Load the weights
weights_df = pd.read_csv(weights_csv_path)
weights = weights_df['Weight'].values

# Step 1: Load and Standardize Column Names
feature_sets = []
meta_data = None  # To store 'filename' and 'category' once, assuming they are consistent

# Load and preprocess feature sets
for path in tqdm(file_paths, desc="Loading Feature Sets", unit="file"):
    df = pd.read_csv(path)
    
    # Rename columns for consistency
    df.rename(columns={'Filename': 'filename', 'Labels': 'category'}, inplace=True)
    
    # Separate metadata (filename and category) from features
    if meta_data is None:
        meta_data = df[['filename', 'category']]
    else:
        df = df.drop(columns=['filename', 'category'], errors='ignore')
    
    # Ensure all feature columns are numeric
    df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
    
    # Add to feature sets list
    feature_sets.append(df)

# Step 2: Apply Weights and Merge Feature Sets
weighted_features = None

for i, (features, weight) in tqdm(enumerate(zip(feature_sets, weights)), desc="Applying Weights", total=len(feature_sets)):
    weighted_set = features.add_prefix(f"set{i}_") * weight  # Apply weight
    weighted_features = pd.concat([weighted_features, weighted_set], axis=1) if weighted_features is not None else weighted_set

# Add labels back to the dataset
final_dataset = pd.concat([meta_data, weighted_features], axis=1)

# Step 3: Prepare Data for Training
labels = final_dataset['category']
features = final_dataset.drop(columns=['filename', 'category'], errors='ignore')

# Encode labels
labels = pd.factorize(labels)[0]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Step 4: Train a Random Forest Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate the Model on Training and Testing Sets
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Training Accuracy: {train_accuracy:.4f}")

# Testing accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Testing Accuracy: {test_accuracy:.4f}")

# Step 6: Detailed Performance on Testing Set
print("\nClassification Report (Testing Set):")
print(classification_report(y_test, y_test_pred))
